In [7]:
import pandas
import requests
from bs4 import BeautifulSoup
import os
import nltk
import pyphen
import re
import openpyxl


nltk.download('punkt')
output_file_path = "./Output Data Structure.xlsx"
sheet_name = 'Sheet1'
pronoun_pattern = r"\b(I|me|my|mine|we|us|our|ours|you|your|yours|he|him|his|she|her|hers|it|its|they|them|their|theirs|myself|ourselves|yourself|yourselves|himself|herself|itself|themselves)\b"


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\navee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
def add_data_to_cell(output_file_path, sheet_name, row_index, column_index, new_data):
    workbook = openpyxl.load_workbook(output_file_path)
    sheet = workbook[sheet_name]
    sheet.cell(row=row_index, column=column_index).value=new_data
    workbook.save(output_file_path)
def read_cell_value(output_file_path, sheet_name, row_index, column_index):
    workbook = openpyxl.load_workbook(output_file_path)
    sheet = workbook[sheet_name]
    cell_value = sheet.cell(row=row_index, column=column_index).value
    return cell_value
def count_syllables(word):
    dic = pyphen.Pyphen(lang='en')
    syllables = dic.inserted(word).count('-') + 1
    return syllables
def count_personal_pronouns(text):
    
    matches = re.findall(pronoun_pattern, text, flags=re.IGNORECASE)
    return len(matches)

In [9]:
folder_path = "./stop_words" 
stop_words=[]
for filename in os.listdir(folder_path):
    file_path2 = os.path.join(folder_path, filename)

    if os.path.isfile(file_path2):
        with open(file_path2, 'r') as file:
            words_to_remove = file.read().splitlines()
            for i in words_to_remove:
                x=i.split('|')
                for j in x:
                    stop_words.append(j.strip())

In [10]:
with open("./master_dictionary/positive-words.txt",'r') as file:
    positive_words=file.read().splitlines()

with open("./master_dictionary/negative-words.txt",'r') as file:
    negative_words=file.read().splitlines()

In [12]:
for row_index in range(2,116):
    column_index = 2 
    #reading
    url= read_cell_value(output_file_path, sheet_name, row_index, column_index)
    print(url)
    page=requests.get(url)

    soup=BeautifulSoup(page.content,'html.parser')
    number_of_words=0
    no_of_char=0
    if soup.h1 is not None:
        heading=soup.h1.text
        paragraphs=[]
        list_of_words=[]
        paragraphs.append(heading)
        results=soup.find_all('p')[16:]
        for para in results:
            paragraphs.append(para.text.replace('.', '').replace(',', '').replace('!', '').replace('?', ''))
            number_of_words+=len(para.text.split())
            no_of_char+=len(str(para))-len(para.text.split())-1
            for word in para.text.split():
                list_of_words.append(word)
        len_of_paragraph=len(paragraphs)
        modified_paragraphs = []
        for paragraph in paragraphs:
            words= paragraph.split()
            filtered_words = [word for word in words if word not in stop_words]
            modified_paragraph = ' '.join(filtered_words)
            modified_paragraphs.append(modified_paragraph)

        final_paragraph=[]
        for paragraph in modified_paragraphs:
            word=nltk.word_tokenize(paragraph)
            for single_word in word:
                final_paragraph.append(single_word)
        edited_length=len(final_paragraph)


        positive_score=0
        negative_score=0
        for paragraph in modified_paragraphs:
            words=paragraph.split()
            for word in words:
                if word in positive_words:
                    positive_score+=1
                elif word in negative_words:
                    negative_score+=1

        polarity_score=(positive_score-negative_score)/((positive_score+negative_score)+0.000001)

        subjectivity_score=(positive_score+negative_score)/(len(final_paragraph)+0.000001)
                
        no_of_words=len(list_of_words)

        list_of_sentences=[]
        list_of_sentences.append(heading)
        
        for para in paragraphs:
            x=para.split('.')
            for i in x:
                list_of_sentences.append(i)
        no_of_sentences=len(list_of_sentences)

        average_sentence_length=no_of_words/no_of_sentences

        complex_word_count=0
        sum_of_syllable_count=0
        for word in list_of_words:
            syllable_count = count_syllables(word)
            sum_of_syllable_count+=syllable_count
            if syllable_count>2:
                complex_word_count+=1
        percentage_of_complex_words=complex_word_count/no_of_words

        fog_index = 0.4 * (average_sentence_length + percentage_of_complex_words)

        average_of_words_per_sentence=no_of_words/no_of_sentences

        word_count=abs(no_of_words-edited_length)

        sylable_count_per_word=sum_of_syllable_count/no_of_words

        pronoun_count=0
        for sentence in list_of_sentences:
            count = count_personal_pronouns(sentence)
            pronoun_count+=count

        average_word_length=no_of_char/no_of_words  
        values_to_save=[positive_score,negative_score,polarity_score,subjectivity_score,average_sentence_length,percentage_of_complex_words,fog_index,average_of_words_per_sentence,complex_word_count,word_count,sylable_count_per_word,pronoun_count,average_word_length]    
        k=0
        for j in range(3,16):

            column_index1 =j 
            new_data = values_to_save[k]

            add_data_to_cell(output_file_path, sheet_name, row_index, column_index1, new_data)
            k+=1
        print('values written at row index ',row_index)
    else:
        print('cannot find row index ',row_index)

https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/
values written at row index  2
https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/
values written at row index  3
https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/
values written at row index  4
https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/
values written at row index  5
https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/
values written at row index  6
https://insights.blackcoffer.com/man-and-machines-together-machines-are-more-diligent-than-humans-blackcoffe/
values written at row index  7
https://insights.blackcoffer.com/in-future-or-in-upcoming-years-humans-and-machines-are-going-to-work-together-in-every-field-of-work/
values written at row index  8
https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
cannot find row index  9
https://i

values written at row index  69
https://insights.blackcoffer.com/impact-of-covid-19-pandemic-on-office-space-and-co-working-industries/
values written at row index  70
https://insights.blackcoffer.com/contribution-of-handicrafts-visual-arts-literature-in-the-indian-economy/
values written at row index  71
https://insights.blackcoffer.com/how-covid-19-is-impacting-payment-preferences/
values written at row index  72
https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-2/
values written at row index  73
https://insights.blackcoffer.com/lessons-from-the-past-some-key-learnings-relevant-to-the-coronavirus-crisis/
values written at row index  74
https://insights.blackcoffer.com/covid-19-how-have-countries-been-responding/
values written at row index  75
https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry-2/
values written at row index  76
https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-3/
values written at row in